# Introduction to Pathogen.jl

## Initialization

We start by loading Pathogen.jl in julia 1.0

In [1]:
using CSV, Distributed, DelimitedFiles, Distances, LinearAlgebra, Statistics, Plots, Random, Distributions, DataFrames, Pathogen

┌ Info: Recompiling stale cache file /Users/justin/.julia/compiled/v1.0/Pathogen/08VJn.ji for Pathogen [58f1fdb4-9bff-11e8-2091-99b816fb7d3c]
└ @ Base loading.jl:1187


Additionally we'll use the core Packages:
* `Random` for setting the random seed for replicability
* `Distances` for various efficienct distance calculation methods
* `DataFrames` for storing individual level information
* `Distributions` for random generation of individual level information
* `Plots` for visualization

## Simulation
We'll set the seed for random number generation such that our results here are reproducible:

In [ ]:
Random.seed!(5432)

We'll define a population consisting of *n* individuals, uniformaly distributed over a 10x10 unit area. For our population we'll assume to have information on a single risk factor, which we'll distribute following a Gamma(1,1) distribution.

With Pathogen.jl, the `Population` type contains all population information, it consists of:
* `risks` - an optional `DataFrame` with all individual level information (risk factors, location)
* `distances` - an optional Array{T<:Any, 2} with precalculated distances
* `individuals` - the number of individuals in the population

In [ ]:
scenario = 4
if scenario in [1; 2]
    n = 25
elseif scenario in [3; 4]
    n = 100
end
    
risks = DataFrame(x = rand(Uniform(0, 10), n),
                  y = rand(Uniform(0, 10), n),
                  riskfactor1 = rand(Gamma(), n))

pop = Population(risks)

# Will precalculate distances
pop.distances = [euclidean([risks[:x][i]; risks[:y][i]], [risks[:x][j]; risks[:y][j]]) for i = 1:pop.individuals, j = 1:pop.individuals]


Pathogen.jl supports SEIR, SEI, SIR, and SI individual level models. Within each of those model types, the user has full control over the functions describing individual specific transition rates between disease states (*i.e.* form of, and relevant risk factors to, susceptibility, transmissibility, transmissability, latency, removal, and sparks functions when applicable). For ease of use, some common forms of these functions are provided in `examples/risk_functions.jl` which can either be loaded and used directly, or serve as examples to help the user construct their own. For this example we will use some of these functions directly in an SEIR individual level model. 

In [ ]:
function _one(params::Vector{Float64}, pop::Population, i::Int64)
  return 1.0
end

function _constant(params::Vector{Float64}, pop::Population, i::Int64)
  return params[1]
end

function _powerlaw(params::Vector{Float64}, pop::Population, i::Int64, k::Int64)
  α = params[1]
  β = params[2]
  d = pop.distances[k, i]
  return α * (d^(-β))
end

rf = RiskFunctions{SEIR}(_constant, # sparks function
                         _one, # susceptibility function
                         _powerlaw, # transmissability function
                         _one, # transmissibility function
                         _constant, # latency function
                         _constant) # removal function

Each of these functions is parameterized by a `Vector{Float64}`...

In [ ]:
if scenario in [1; 3]
    rparams = RiskParameters{SEIR}([0.0001], # sparks function parameter(s)
                                   Float64[], # susceptibility function parameter(s)
                                   [0.5, 3.0], # infectivity function parameter(s)
                                   Float64[], # transmissibility function parameter(s)
                                   [0.1], # latency function parameter(s)
                                   [0.05]) # removal function parameter(s)
elseif scenario in [2; 4]
    rparams = RiskParameters{SEIR}([0.0001], # sparks function parameter(s)
                               Float64[], # susceptibility function parameter(s)
                               [1.0, 5.0], # infectivity function parameter(s)
                               Float64[], # transmissibility function parameter(s)
                               [0.1], # latency function parameter(s)
                               [0.05]) # removal function parameter(s)
end

We initialize an Epidemic `Simulation` object by supplying the population `DataFrame` with our individual level model risk functions and their associated parametrizations.

In [ ]:
starting_states = append!([State_I], fill(State_S, n-1)) # Set first individual as infectious, others as susceptible to start
sim = Simulation(pop, starting_states, rf, rparams)

This simulation can now be ran until either a specified stop condition is met (computation time, simulation time, or a maximum number of iterations), or until there are no further possible events.

In [ ]:
simulate!(sim, tmax=100.0)

## Simulation Visualization
Using our preferred plotting backend we can produce several informative plots for a simulated epidemic...

In [ ]:
gr()

a `plot()` method is provided for `Events` objects - such as those generated with an epidemic simulation. This will plot an epidemic curve.

In [ ]:
plot(sim.events, 0.0, 100.0)
png(joinpath(@__DIR__, "plots/epidemic_curve_$scenario-$core.png"))

We can also visualize transmission networks through time - below is an animation showing the spread of an epidemic for the first 100 time units.

In [ ]:
epidemic_animation = @animate for t in range(0.0, stop=100.0, length=100) # From t = 0.0 to t= 100.0 with 100 increments
    plot(sim.transmission_network, sim.population, sim.events, t)
end
mp4(epidemic_animation, joinpath(@__DIR__, "plots/epidemic_animation_$scenario-$core.mp4"), fps=10)

## Inference

In [ ]:
# Generate observations with Uniform(0, 2) observation delay for infection and removal
obs = observe(sim, Uniform(0.0, 2.0), Uniform(0.0, 2.0))

In [ ]:
# Optimistically assume we know the functional form of epidemic (i.e. use same risk functions used for simulation purposes)
# Specify some priors for the risk parameters of our various risk functions
# Set some extents for event data augmentation

rpriors = RiskPriors{SEIR}([Uniform(0.0, 0.01)],
                           UnivariateDistribution[],
                           [Uniform(0.0, 2.0); Uniform(1.0, 8.0)],
                           UnivariateDistribution[],
                           [Uniform(0.0, 1.0)],
                           [Uniform(0.0, 1.0)])

ee = EventExtents{SEIR}(20.0, 5.0, 5.0)

In [ ]:
# Initialize MCMC
mcmc = MCMC(obs, ee, pop, rf, rpriors)
start!(mcmc, 1, attempts=20000) # 1 chain, with 20k initialization attempts each

In [ ]:
# Run MCMC
start_time = time()
iterate!(mcmc, 50000, 0.5, event_batches=20)
mcmc_time = start_time - time()
writecsv(joinpath(@__DIR__, "data/mcmc_time_$scenario-$core.mp4.csv"), convert(Float64, mcmc_time))

In [ ]:
plot(mcmc.markov_chains[1].risk_parameters, y_scale=:log10)
png(joinpath(@__DIR__, "plots/parameter_trace_$scenario-$core.png"))

In [2]:
function mean(networkvector::Vector{TransmissionNetwork})
  return mean([vcat(networkvector[i].external', networkvector[i].internal) for i in 1:length(networkvector)])
end

mean (generic function with 1 method)

In [ ]:
    # Prepare to export results
    network_mean = mean(mcmc.markov_chains[1].transmission_network[5001:25000])
    network_MSE = sum((network_mean .- vcat(sim.transmission_network.external', sim.transmission_network.internal)).^2)/prod(size(network_mean))
    tracedata = Array(mcmc.markov_chains[1].risk_parameters)
    events_trace = convert(Array{Float64, 2}, mcmc.markov_chains[1].events)
    events_mean = mean(mcmc.markov_chains[1].events)
    events_actual = convert(Vector{Float64}, sim.events)

    writecsv(joinpath(@__DIR__, "data/network_mse_$scenario-$core.csv"), network_MSE)
    writecsv(joinpath(@__DIR__, "data/parameter_trace_$scenario-$core.csv"), tracedata)
    writecsv(joinpath(@__DIR__, "data/events_trace_$scenario-$core.csv"), events_trace)
    writecsv(joinpath(@__DIR__, "data/events_mean_$scenario-$core.csv"), events_actual)
    writecsv(joinpath(@__DIR__, "data/events_actual_$scenario-$core.csv"), events_actual)
